In [1]:
import torch
from torch import Tensor, nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import dataset
from torch.utils.tensorboard import SummaryWriter

import regex as re
import os
import time
from tqdm import tqdm
import copy
import math

from model import TransformerModel
from utils import preProcessText, getTokenizer
from config import getConfig

In [2]:
def get_model(model_config, ntokens):
    emsize = model_config["emsize"]
    d_hid = model_config["d_hid"]
    nlayers = model_config["nlayers"]
    nhead = model_config["nhead"]
    dropout = model_config["dropout"]
    model = TransformerModel(ntokens, emsize,nhead, d_hid, nlayers, dropout)
    return model

def loadModelExceptDecoderWeight(model, best_model_path):
    if os.path.exists(best_model_path):
        model_state_dict = model.state_dict()
        
        print(f"Preloading model {best_model_path}")
        state = torch.load(best_model_path)
        pretrained_state_dict = state['model_state_dict']
        pretrained_state_dict = {k: v for k, v in pretrained_state_dict.items() if 'decoder' not in k}

        model_state_dict.update(pretrained_state_dict)
        model.load_state_dict(model_state_dict)

        for param in model.decoder.parameters():
            param.requires_grad = True
        
        return model
    else:
        raise Exception("Model Not Found")

In [3]:
model_config, app_config = getConfig()
print(model_config)
print(app_config)

bptt=model_config["bptt"]

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

softmax = nn.Softmax(dim=2)

tokenizer, vocab = getTokenizer()
ntokens = len(vocab)
model = get_model(model_config, ntokens).to(device)

criterion = nn.CrossEntropyLoss()
lr = 1  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

{'emsize': 300, 'd_hid': 1024, 'nlayers': 4, 'nhead': 4, 'dropout': 0.2, 'bptt': 64}
{'logs': 'tensorboard_logs', 'epochs': 25}
cuda


/home/niranjan/miniconda3/envs/cslr/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [4]:
print(model)

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=1024, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=1024, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (embedding): Embedding(60507, 300)
  (decoder): Linear(in_features=300, out_features=60507, bias=True)
)


In [5]:
best_model_path = 'models/best_model_sample_test_corrected.pt'
loaded_model = loadModelExceptDecoderWeight(model, best_model_path)
print(loaded_model)

Preloading model models/best_model_sample_test_corrected.pt
TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=1024, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=1024, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (embedding): Embedding(60507, 300)
  (decoder): Linear(in_features=300, out_features=60507, bias=True)
)
